# Introduction to the biofilter model

In [ ]:
# Date: 26-03-2021
# Creator: Anemoon Drinkwaard

# This model calculates the change in ammonia and methane concentration over the volume of the conventional biofilter.
# Paracoccus is used as the microbe in this version of the model specifically.

# The Paracoccus dnitrificans in this work is a synthetic methanotroph that can perform HNAD (nitrification and denitrification) but cannot perform dinitrogen fixation.
# Therefore, the GMO P. denitrificans needs methane and oxygen for growth.
# If methane, oxygen or ammonia is depleted, there is no activity in that part of the biofilm.

# The model can be ranged over the initial concentrations of methane and ammonia, or over the gas flow rate, (or both, but the results are only displayed in graphs for either of the two).

# First the specific reactions rates are determined with the metabolic model for P. denitrificans (To accurately get the results from the P. denitrificans model without ammonia, the corresponging cell should be run twice)
# Then the specific reaction rates are used to caclulate the change in compound concentration over the volume of the reactor (the simulated reactor volume can be altered within the Biofilter calculation cell in the "linspace" function)
# The concentrations and volumes are used to calculate the penetration depth if the compound is the first limiting compound (if there's only one active layer)
# Finally, the biofilter performance measures removal efficiency and productivity are calculated

# At the end, results displayed in the thesis corresponding to this model can be calculated/made,
# First for ranging initial concentrations, then for the ranging gas flow rate

# Specific parameters can be altered in the Parameter cell
# The model used can be altered in the "Loading GSM and importing modules cell"

# Loading GSMM and importing modules

In [ ]:
import math 
import cobra.test
import os
from os.path import join
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import seaborn as sb
import pandas as pd
from mpl_toolkits import mplot3d

# Loading the SBML model
data_dir = cobra.test.data_dir
from cobra.io import read_sbml_model
gsmm = cobra.io.read_sbml_model(join(data_dir,"Paracoccus_MinimalMedia_sMMO.sbml"))

# List of parameters

In [ ]:
# Packing material parameters
ep = 0.5 # porosity of the particles
phi = 1 # sphericity
tp = 1.23*((1-ep)**(4/3)/(ep*phi)) # tortuosity of the aggregates

# General diffusion coefficients (m2/h)
Ds = 5.36e-6 # methane
Do = 7.56e-6 # oxygen
Dn = 5.90e-6 # ammonia
Dn2 = 6.77e-6 # dinitrogen

# Effective diffusion coefficients in a particle (m2/h)
Dsp = Ds * (ep / tp) # methane
Dop = Do * (ep / tp) # oxygen
Dnp = Dn * (ep / tp) # ammonia
Dn2p = Dn2 * (ep / tp) # dinitrogen

# Dimensionless Henry constants (KHcc)^-1 or partition coefficients
ms = 28.8 # methane
mo = 31.0 # oxygen
mn = 0.000733 # ammonia
mn2 = 63.0 # dinitrogen

# Gas mass transfer coefficients (1/h)
ksga = 1300# 300 # methane
knga = 1400 #275 # ammonia

# Biofilm parameters
Cxp = 1000 # cell concentration in a particle (gDW/m3)
L = 2.5e-3 # total depth of the biofilm (m)
dl = 1e-4 # thickness of the liquid stagnant layer (m)
a = 500 # specific surface area (m^-1)
FG = [12000] # gas flowrate (m3/h) >> Give a range or single value in an array: [2400, 4800, 12000, 30000, 60000]

# initial gas concentrations (mmol/m3):
CSG0 = [4.09, 10.2, 20.4, 30.7, 40.9] # methane >> Give a range or single value in an array: [4.09, 10.2, 20.4, 30.7, 40.9]
Cog0 = 8.57e3 #oxygen 
CNG0 = [0.409,  1.02, 2.04, 3.07, 4.09] # ammonia Give a range or single value in an array: [0.409,  1.02, 2.04, 3.07, 4.09]
Cn2g0 = 3.19e4 #dinitrogen         

# Determining specific reaction rates

In [ ]:
# Determination of the uptake rates when methane, oxygen, ammonia and dinitrogen are present

# If the methane uptake rate constrain should be altered, open the next line of code:
#gsmm.reactions.get_by_id('EX_cpd01024_e0').lower_bound = -184.6

# by executing FBA on the GSMM
gsmm1 = gsmm
sol_1 = gsmm1.optimize()

# Calling the specifc fluxes of methane, oxygen, ammonia and dinitrogen exchange
qs_1 = -gsmm1.reactions.get_by_id('EX_cpd01024_e0').flux
qo_1 = -gsmm1.reactions.get_by_id('EX_cpd00007_e0').flux
qn_1 = -gsmm1.reactions.get_by_id('EX_cpd00013_e0').flux
qn2_1 = -gsmm1.reactions.get_by_id('EX_cpd00528_e0').flux
mu_1 = gsmm1.reactions.get_by_id('bio1').flux

print("qs_1:", qs_1, "qo_1:", qo_1, "qn_1:", qn_1, "qn2_1:", qn2_1, "mu_1:", mu_1) # in mmol/gDW/h


In [ ]:
# Determination of the uptake rates when methane, oxygen, and dinitrogen are present (ammonia is depleted)
# To accurately get the results from the P. denitrificans model used in this model, this cell should be run twice.

# Adjusting the media in the model after the depletion of ammonia
gsmm2 = gsmm
medium = gsmm2.medium
medium['EX_cpd00013_e0'] = 0.0 # ammonia > FBA
gsmm2.medium = medium
    
# Execute FBA on the GSMM
sol_2 = gsmm2.optimize()
print(gsmm2.medium)
    
# Calling the specifc fluxes of methane, oxygen, ammonia and dinitrogen exchange
qs_2 = -gsmm2.reactions.get_by_id('EX_cpd01024_e0').flux
qo_2 = -gsmm2.reactions.get_by_id('EX_cpd00007_e0').flux
qn_2 = -gsmm2.reactions.get_by_id('EX_cpd00013_e0').flux
qn2_2 = -gsmm2.reactions.get_by_id('EX_cpd00528_e0').flux
mu_2 = gsmm2.reactions.get_by_id('bio1').flux
print(gsmm2.summary())
    
print("qs_2:", qs_2, "qo_2:", qo_2, "qn_2:", qn_2, "qn2_2:", qn2_2, "mu_2:", mu_2) # in mmol/gDW/h

# Biofilter calculations

In [ ]:
# Setting up empty data arrays per series to save final calculated outputs in
data_Cg = []
data_Clp = []
data_L_1 = []
data_P = []
data_E = []
data_Dg = []

# Loop for range of gas flow rate values
for i in FG:
    Fg = i
    
    # Loop for range of initial methane concentrations
    for j in CSG0:
        Csg0 = j
        
        # Loop for range of inirial ammonia concentrations
        for k in CNG0:
            Cng0 = k

            # Gas microbalances methane,and ammonia
            def model(Cg,V):
                Csg = Cg[0]
                Cng = Cg[1]

                # Balances for when ammonia is depleted
                if Cng < 0:
                    # Situation if the microbe can fix dinitrogen
                    if qs_2 != 0:
                        s = ((-(dl/(Ds*a) + 1/ksga) + math.sqrt((dl/(Ds*a) + 1/ksga)**2 + 4*(Csg/ms)*(math.sqrt(2*Dsp*qs_2*Cxp)*a)**(-2))) / (Fg*2*(math.sqrt(2*Dsp*qs_2*Cxp)*a)**(-2)))

                        dCsgdV = -min(qs_2*Cxp*L*a/Fg, s)
                        dCngdV = 0
                    # Situation if the microbe cannot fix dinitrogen
                    else:
                        dCsgdV = 0
                        dCngdV = 0

                # Balances for when ammonia is not depleted
                else:
                    s = ((-(dl/(Ds*a) + 1/ksga) + math.sqrt((dl/(Ds*a) + 1/ksga)**2 + 4*(Csg/ms)*(math.sqrt(2*Dsp*qs_1*Cxp)*a)**(-2))) / (Fg*2*(math.sqrt(2*Dsp*qs_1*Cxp)*a)**(-2)))
                    n = ((-(dl/(Dn*a) + 1/knga) + math.sqrt((dl/(Dn*a) + 1/knga)**2 + 4*(Cng/mn)*(math.sqrt(2*Dnp*qn_1*Cxp)*a)**(-2))) / (Fg*2*(math.sqrt(2*Dnp*qn_1*Cxp)*a)**(-2)))
                    
                    # Situation if the microbe can fix dinitrogen
                    if qs_2 != 0:
                        # Balances for a situation with two active layers in the biofilm
                        if n/(qn_1*Cxp*a) == min(L, s/(qs_1*Cxp*a), n/(qn_1*Cxp*a)):
                            L1 = n*Fg / (qn_1*Cxp)
                            dCsgdV = -((-(1/ksga + dl/(Ds*a) - ((qs_1-qs_2)*L1)/(a*Dsp*qs_2)) + math.sqrt(((1/ksga + dl/(Ds*a) - ((qs_1-qs_2)*L1)/(a*Dsp*qs_2))**2) + 4*(Csg/ms + ((-qs_1*qs_2 + qs_2**2)*Cxp*(L1**2))/(2*Dsp*qs_2**2))*(1/(2*Dsp*qs_2*Cxp*a**2)))) / (Fg* 2 / (Cxp*2*Dsp*qs_2*a**2)))
                            dCngdV = -n

                        # Balances for a situation with one active layer in the biofilm
                        else:
                            dCsgdV = -min(qs_1*Cxp*L*a/Fg, s)
                            dCngdV = -min(qn_1*Cxp*L*a/Fg, (qn_1/qs_1)*s)
                            
                    # Situation if the microbe cannot fix dinitrogen
                    else:
                        dCsgdV = -min(qs_1*Cxp*L*a/Fg, s)
                        dCngdV = -min(qn_1*Cxp*L*a/Fg, (qn_1/qs_1)*s)
                
                return [dCsgdV, dCngdV]                  

            # Initial conditions >> Cgin (mmol/m3) 
            Cg0 = [Csg0, Cng0]   

            # Spacial points and total Volume simulated for the biofilter (Volume (m3))
            V = np.linspace(0, 150000, 600000) 
            #(0, 25000, 100000) as general values, but higher values (0, 100000, 600000) or (0, 150000, 600000) used for efficiency calculations with different initial concentrations

            # Solve the microbalances and calculate gas concentrations for methane and ammonia
            Cg = odeint(model, Cg0, V)

            Csg = Cg[:,0]
            Cng = Cg[:,1]
            
            # Add gas concentration values (Csg and Cng) to data_Cg array
            data_Cg.append(Csg)
            data_Cg.append(Cng)

            # Calculation of critical particle concentrations methane and ammonia (L=Li) 
            Cslpcrit = (qs_1*Cxp)/(2*Dsp)*(L**2)
            Cnlpcrit = (qn_1*Cxp)/(2*Dnp)*(L**2)
            # Calculation of critical liquid concentrations methane and ammonia (L=Li) 
            Cslcrit = (qs_1*Cxp*L*dl)/Ds + Cslpcrit
            Cnlcrit = (qn_1*Cxp*L*dl)/Dn + Cnlpcrit
            # Calculation of gas concentrations methaneand ammonia (L=Li) 
            Csgcrit = (qs_1*Cxp*L*a*ms)/ksga + Cslcrit*ms
            Cngcrit = (qn_1*Cxp*L*a*mn)/knga + Cnlcrit*mn

            # Creating empty arrays to store particle concentrations
            Cslp = np.zeros(len(Csg))
            Cnlp = np.zeros(len(Csg))
            # Creating empty arrays to store transport rates
            Ns = np.zeros(len(Csg))
            Nn = np.zeros(len(Csg))
            # Creating empty arrays to store penetration depths
            Ls_1 = np.zeros(len(Csg)) 
            Ln_1 = np.zeros(len(Cng))

            # Biofilm concentrations and penetrations depths calculations if methane is the first limiting compound (one active layer)
            for i in range(0,len(Csg)):
                if qs_1 !=0:
                    if Csg[i] >= 10e-20:
                        if Csg[i] >= Csgcrit:
                            Cslp[i] = Cslpcrit
                            Ls_1[i] = L
                        else:
                            if Cng[i] > 10e-12:
                                Ns[i] = ((-(dl/(Ds*a) + 1/ksga) + math.sqrt((dl/(Ds*a) + 1/ksga)**2 + 4*(Csg[i]/ms)*(math.sqrt(2*Dsp*qs_1*Cxp)*a)**(-2))) / (2*(math.sqrt(2*Dsp*qs_1*Cxp)*a)**(-2)))
                                Cslp[i] = -Ns[i]*((dl/(Ds*a) + 1/ksga)) + Csg[i]/ms
                                if Cslp[i] >= 0:
                                    Ls_1[i] = math.sqrt((2 * Dsp * Cslp[i]) / (qs_1 * Cxp))
                                else:
                                    Ls_1[i] = 0
                            else:
                                if qs_2 > 0:
                                    Ns[i] = ((-(dl/(Ds*a) + 1/ksga) + math.sqrt((dl/(Ds*a) + 1/ksga)**2 + 4*(Csg[i]/ms)*(math.sqrt(2*Dsp*qs_2*Cxp)*a)**(-2))) / (2*(math.sqrt(2*Dsp*qs_2*Cxp)*a)**(-2)))
                                    Cslp[i] = -Ns[i]*((dl/(Ds*a) + 1/ksga)) + Csg[i]/ms
                                    if Cslp[i] >= 0:
                                        Ls_1[i] = math.sqrt((2 * Dsp * Cslp[i]) / (qs_2 * Cxp))
                                    else:
                                        Ls_1[i] = 0
                                else: 
                                    Cslp[i] = 0
                                    Ls_1[i] = 0
                    else:
                        Cslp[i] = 0
                        Ls_1[i] = 0
                else:
                    if Csg[i] > 0:
                        Ns[i] = 0
                        Cslp[i] = -Ns[i]*((dl/(Ds*a) + 1/ksga)) + Csg[i]/ms
                        Ls_1[i] = L
                    else: 
                        Cslp[i] = 0
                        Ls_1[i] = 0

            # Biofilm concentrations and penetrations depths calculations if ammonia is the first limiting compound (one active layer)
            for i in range(0,len(Cng)):
                if qn_1 != 0:
                    if Cng[i] >= 0:
                        if Cng[i] >= Cngcrit:
                            Cnlp[i] = Cnlpcrit
                            Ln_1[i] = L
                        else: 
                            Nn[i] = ((-(dl/(Dn*a) + 1/knga) + math.sqrt((dl/(Dn*a) + 1/knga)**2 + 4*(Cng[i]/mn)*(math.sqrt(2*Dnp*qn_1*Cxp)*a)**(-2))) / (2*(math.sqrt(2*Dnp*qn_1*Cxp)*a)**(-2)))
                            Cnlp[i] = -Nn[i]*((dl/(Dn*a) + 1/knga)) + Cng[i]/mn
                            if Cnlp[i] >= 0:
                                Ln_1[i] = math.sqrt((2 * Dnp * Cnlp[i]) / (qn_1 * Cxp))
                            else:
                                Ln_1[i] = 0
                    else:
                        Cnlp[i] = 0
                        Ln_1[i] = 0
                else:
                    if Cng[i] > 0:
                        Nn[i] = 0
                        Cnlp[i] = -Nn[i]*((dl/(Dn*a) + 1/knga)) + Cng[i]/mn
                        Ln_1[i] = L
                    else:
                        Cnlp[i] = 0
                        Ln_1 = 0
                        
            # Add particle concentrations (Clp) and penetration depths (L_1) values for methane and ammonia to data_Clp and data_L_1 array
            data_Clp.append(Cslp)
            data_L_1.append(Ls_1)
            data_Clp.append(Cnlp)
            data_L_1.append(Ln_1)
            
            # Perform sensitivity analysis by calculating productivity and (conversion) efficiency

            # Creating empty arrays to store productivities
            Ps = np.zeros(len(Csg))
            Pn = np.zeros(len(Cng))
            
            # Calculate productivities
            for i in range(0, len(V)): 
                Ps[i] = ((Csg0-Csg[i])*Fg)/V[i]
                Pn[i] = ((Cng0-Cng[i])*Fg)/V[i]
            
            # Add productivity values to data_P array    
            data_P.append(Ps)
            data_P.append(Pn)
 
            # Creating empty arrays to store efficiencies
            Es = np.zeros(len(Csg))
            En = np.zeros(len(Cng))
        
            # Calculate efficiencies
            for i in range (0, len(V)):
                Es[i] = ((Csg0-Csg[i])/ Csg0)*100
                En[i] = ((Cng0-Cng[i])/ Cng0)*100
            
            # Add efficiency values to data_E array
            data_E.append(Es)
            data_E.append(En)
            
            # Calculate gas dilution rate
            Dg = np.zeros(len(V))
            for i in range(0, len(V)):
                Dg[i] = Fg/V[i]
            if len(FG) != 1:
                data_Dg.append(Dg)
                
            # Print function to keep track of the loops while the code in running
            print("loop complete: Fg=", Fg, "Csg0=", Csg0, "Cng0=", Cng0)
            

# Plotting results: constant gas flow rate, ranging initial conditions

In [ ]:
# Direct results of the model: initial concentrations vs reactor volume

# These results are only calculated if one value for FG is given
if len(FG) == 1:
    
    # Styling of the graphs
    sb.set(font_scale=1.2, style="white")

    # plot the results: Csg (methane inital concentration)
    plt.plot(V,data_Cg[0],'b-', label='Ratio 10:1')
    plt.plot(V,data_Cg[10],'r-.', label='Ratio 25:1')
    plt.plot(V,data_Cg[20],'g--', label='Ratio 50:1')
    plt.plot(V,data_Cg[30],'k:', label='Ratio 75:1')
    plt.plot(V,data_Cg[40],'y-', label='Ratio 100:1')
    plt.ylabel('Csg(V)')
    plt.xlabel('V')
    plt.xlim(0,25000)
    plt.legend()
    plt.show()

    # plot the results: Cng (ammonia initial concentration)
    plt.plot(V,data_Cg[1],'b-', label='Ratio 10:1')
    plt.plot(V,data_Cg[11],'r-.', label='Ratio 25:1')
    plt.plot(V,data_Cg[21],'g--', label='Ratio 50:1')
    plt.plot(V,data_Cg[31],'k:', label='Ratio 75:1')
    plt.plot(V,data_Cg[41],'y-', label='Ratio 100:1')
    plt.ylabel('Cng(V)')
    plt.xlabel('V')
    plt.xlim(0,25000)
    plt.legend()
    plt.show()
    
else:
    print("FG has multiple values in its array")

In [ ]:
# Results sensitivity analysis: Methane

# These results are only calculated if one value for FG is given
if len(FG) == 1:
    
    # Styling of the graphs
    sb.set(font_scale=1.2, style="whitegrid")

    # plot the results: Ps vs Dg (methane productivity vs dilution rate)
    plt.plot(Dg,data_P[0],'b-.', label='Csg:4.09')
    plt.plot(Dg,data_P[10],'k-', label='Csg:10.2')
    plt.plot(Dg,data_P[20],'b--', label='Csg:20.4')
    plt.plot(Dg,data_P[30],'k:', label='Csg:30.7')
    plt.plot(Dg,data_P[40],'b-', label='Csg:40.9')
    plt.ylabel('Ps')
    plt.xlabel('Dg')
    plt.xlim(0,10)
    plt.legend()
    plt.show()

    # plot the results: Es vs Dg (methane efficiency vs dilution rate)
    plt.plot(Dg,data_E[0],'k-')
    plt.plot(Dg,data_E[10],'r-.')
    plt.plot(Dg,data_E[20],'g--')
    plt.plot(Dg,data_E[30],'k:')
    plt.plot(Dg,data_E[40],'y-')
    plt.ylabel('η')
    plt.xlabel('Dg')
    plt.xlim(0,10)
    plt.legend()
    plt.show()
    
else:
    print("FG has multiple values in its array")

In [ ]:
# Results sensitivity analysis: ammonia

# These results are only calculated if one value for FG is given
if len(FG) == 1:

    # Styling of the graphs
    sb.set(font_scale=1.2, style="whitegrid")
    
    # plot the results: Pn vs Dg (ammonia productivity vs dilution rate)
    plt.plot(Dg,data_P[9],'r-.', label='Csg:4.09')
    plt.plot(Dg,data_P[19],'k-', label='Csg:10.2')
    plt.plot(Dg,data_P[29],'r--', label='Csg:20.4')
    plt.plot(Dg,data_P[39],'k:', label='Csg:30.7')
    plt.plot(Dg,data_P[49],'r-', label='Csg:40.9')
    plt.xlim(0,10)
    plt.ylabel('Pn')
    plt.xlabel('Dg')
    #plt.legend()
    plt.show()

    # plot the results: En vs Dg (ammonia efficiency vs dilution rate)
    plt.plot(Dg,data_E[21],'k-')
    plt.plot(Dg,data_E[11],'r-.')
    plt.plot(Dg,data_E[21],'g--')
    plt.plot(Dg,data_E[31],'k:')
    plt.plot(Dg,data_E[41],'y-')
    plt.xlim(0,10)
    plt.ylabel('η')
    plt.xlabel('Dg')
    plt.legend()
    plt.show()

else:
    print("FG has multiple values in its array")

In [ ]:
# 3D plot of ammonia productivity vs dilution rate vs ratio methane:ammonia inital concentrations

# These results are only calculated if one value for FG is given
if len(FG) == 1:

    # Set-up of the plot/figure
    sb.set(font_scale=1.2, style="white")
    fig = plt.figure()
    ax = plt.gca(projection='3d')

    #list of all different graphs (graph turned off by '#' are a double ratio)
    x = Dg[4800:100000]
    y = data_P[1][4800:100000]
    ax.plot(x, y, zs=4.09, color='k', zdir="y")

    x = Dg[4800:100000]
    y = data_P[3][4800:100000]
    ax.plot(x, y, zs=4.09, color='k', zdir="y")

    x = Dg[4800:100000]
    y = data_P[5][4800:100000]
    ax.plot(x, y, zs=4.09, color='k', zdir="y")

    x = Dg[4800:100000]
    y = data_P[7][4800:100000]
    ax.plot(x, y, zs=4.09, color='k', zdir="y")

    x = Dg[4800:100000]
    y = data_P[9][4800:100000]
    ax.plot(x, y, zs=4.09, color='k', zdir="y")

    x = Dg[4800:100000]
    y = data_P[11][4800:100000]
    ax.plot(x, y, zs=10.2, color='r', zdir="y")

    x = Dg[4800:100000]
    y = data_P[13][4800:100000]
    ax.plot(x, y, zs=10.2, color='r', zdir="y")

    x = Dg[4800:1000001]
    y = data_P[15][4800:100000]
    ax.plot(x, y, zs=10.2, color='r', zdir="y")

    x = Dg[4800:100000]
    y = data_P[17][4800:100000]
    ax.plot(x, y, zs=10.2, color='r', zdir="y")

    x = Dg[4800:100000]
    y = data_P[19][4800:100000]
    ax.plot(x, y, zs=10.2, color='r', zdir="y")

    x = Dg[4800:100000]
    y = data_P[21][4800:100000]
    ax.plot(x, y, zs=20.4, color='k', zdir="y")

    x = Dg[4800:100000]
    y = data_P[23][4800:100000]
    ax.plot(x, y, zs=20.4, color='k', zdir="y")

    x = Dg[4800:100000]
    y = data_P[25][4800:100000]
    ax.plot(x, y, zs=20.4, color='k', zdir="y")

    x = Dg[4800:100000]
    y = data_P[27][4800:100000]
    ax.plot(x, y, zs=20.4, color='k', zdir="y")

    x = Dg[4800:100000]
    y = data_P[29][4800:100000]
    ax.plot(x, y, zs=20.4, color='k', zdir="y")

    x = Dg[4800:100000]
    y = data_P[31][4800:100000]
    ax.plot(x, y, zs=30.7, color='r', zdir="y")

    x = Dg[4800:100000]
    y = data_P[33][4800:100000]
    ax.plot(x, y, zs=30.7, color='r', zdir="y")

    x = Dg[4800:100000]
    y = data_P[35][4800:100000]
    ax.plot(x, y, zs=30.7, color='r', zdir="y")

    x = Dg[4800:100000]
    y = data_P[37][4800:100000]
    ax.plot(x, y, zs=30.7, color='r', zdir="y")

    x = Dg[4800:100000]
    y = data_P[39][4800:100000]
    ax.plot(x, y, zs=30.7, color='r', zdir="y")

    x = Dg[4800:100000]
    y = data_P[41][4800:100000]
    ax.plot(x, y, zs=40.9, color='k', zdir="y")

    x = Dg[4800:100000]
    y = data_P[43][4800:100000]
    ax.plot(x, y, zs=40.9, color='k', zdir="y")

    x = Dg[4800:100000]
    y = data_P[45][4800:100000]
    ax.plot(x, y, zs=40.9, color='k', zdir="y")

    x = Dg[4800:100000]
    y = data_P[47][4800:100000]
    ax.plot(x, y, zs=40.9, color='k', zdir="y")

    x = Dg[4800:100000]
    y = data_P[49][4800:100000]
    ax.plot(x, y, zs=40.9, color='k', zdir="y", )

    # Set axes of the 3D plot
    ax.view_init(25, -110)
    ax.set_xlabel('Dg')
    ax.set_ylabel('Csg')
    ax.set_zlabel('En')

    plt.show()

else:
    print("FG has multiple values in its array")

In [ ]:
# Heat maps of maximum theoretical achievable Es (methane efficiency) and En (ammonia efficiency) against Cng (ammonia) and Csg (methane initial concentreation)

# These results are only calculated if one value for FG is given
if len(FG) == 1:
    
    # Setting up empty arrays 
    E_s = []
    E_n = []
    V_t = []

    # Find maximum achievable Es
    for i in range(0, 49, 2):
        E_s.append(np.amax(data_E[i]))

    # Find maximum achievable En
    for i in range(1, 50, 2):
        E_n.append(np.amax(data_E[i]))

    # Create dataframe for max Es and En
    df_En = pd.DataFrame(data={CSG0[0] : E_n[0:5], CSG0[1] : E_n[5:10], CSG0[2] : E_n[10:15], CSG0[3] : E_n[15:20], CSG0[4] : E_n[20:25]}, index=CNG0)
    df_Es = pd.DataFrame(data={CSG0[0] : E_s[0:5], CSG0[1] : E_s[5:10], CSG0[2] : E_s[10:15], CSG0[3] : E_s[15:20], CSG0[4] : E_s[20:25]}, index=CNG0)

    df_En = df_En.astype(float)
    df_Es = df_Es.astype(float)

    # Automatically create het maps for maximum Es and En
    sb.set(font_scale=1.2)
    sb.heatmap(df_En, annot=True, fmt='.0f', cbar_kws={"label": "En"})
    plt.xlabel("Csg")
    plt.ylabel("Cng")
    plt.show()

    sb.heatmap(df_Es, annot=True, cmap="YlGnBu_r", fmt='.0f', cbar_kws={"label": "Es"})
    plt.xlabel("Csg")
    plt.ylabel("Cng")
    plt.show()

else:
    print("FG has multiple values in its array")

In [ ]:
# Heat maps of Ps (methane productivity), Pn (ammonia productivity), V (reactor volume) and En (ammonia efficiency)
# against Csg (methane) and Cng (methane initial concentration) ratio >> based on different values of Es (methane efficiency)

# These results are only calculated if one value for FG is given
if len(FG) == 1:

    Es = [80, 60, 40] # array of baseline Es
    # Setting up empty arrays
    Ps_E = []
    Pn_E = []
    V_E = []
    En_E = []

    # Calculate the V at the specific Es; with this V calculate Ps, Pn and En, for cases all three Es can be reached
    for i in [8, 6, 4, 18, 16, 2, 14, 26, 38]:
            Es_func = interp1d(data_E[i], V)
            VE = Es_func(Es)
            V_E.append(VE/1e4)

            Ps_func = interp1d(V, data_P[i])
            PsE = Ps_func(VE)
            Ps_E.append(PsE)

            Pn_func = interp1d(V, data_P[i+1])
            PnE = Pn_func(VE)
            Pn_E.append(PnE)

            En_func = interp1d(V, data_E[i+1])
            EnE = En_func(VE)
            En_E.append(EnE)
    
    # Calculate the V at the specific Es; with this V calculate Ps, Pn and En, for cases 40 and 60% Es can be reached
    for i in [24]:
        for j in Es:
            if j == 80:
                V_E.append(np.nan)
                Ps_E.append(np.nan)
                Pn_E.append(np.nan)
                En_E.append(np.nan)
            else:    
                Es_func = interp1d(data_E[i], V)
                VE = Es_func(j)
                V_E.append(VE/1e4)

                Ps_func = interp1d(V, data_P[i])
                PsE = Ps_func(VE)
                Ps_E.append(PsE)

                Pn_func = interp1d(V, data_P[i+1])
                PnE = Pn_func(VE)
                Pn_E.append(PnE)

                En_func = interp1d(V, data_E[i+1])
                EnE = En_func(VE)
                En_E.append(EnE)
    
    # Calculate the V at the specific Es; with this V calculate Ps, Pn and En, for cases all three40% Es can be reached
    for i in [46, 34]:
        for j in Es:
            if j == 40:
                Es_func = interp1d(data_E[i], V)
                VE = Es_func(j)
                V_E.append(VE/1e4)

                Ps_func = interp1d(V, data_P[i])
                PsE = Ps_func(VE)
                Ps_E.append(PsE)

                Pn_func = interp1d(V, data_P[i+1])
                PnE = Pn_func(VE)
                Pn_E.append(PnE)

                En_func = interp1d(V, data_E[i+1])
                EnE = En_func(VE)
                En_E.append(EnE)
            else:
                V_E.append(np.nan)
                Ps_E.append(np.nan)
                Pn_E.append(np.nan)
                En_E.append(np.nan)

    # Creating a dataframe out of Pn, Ps, V and En
    df_Pn = pd.DataFrame(data={1 : Pn_E[0], 1.33 : Pn_E[1], 2 : Pn_E[2],  4 : Pn_E[5], 2.5 : Pn_E[3], 3.33 : Pn_E[4], 5 : Pn_E[6], 6.67 : Pn_E[7], 10 : Pn_E[9:12], 7.5 : Pn_E[8], 15 : Pn_E[15:18], 13.33 : Pn_E[12:15]}, index=Es)
    df_Ps = pd.DataFrame(data={1 : Ps_E[0], 1.33 : Ps_E[1], 2 : Ps_E[2],  4 : Ps_E[5], 2.5 : Ps_E[3], 3.33 : Ps_E[4], 5 : Ps_E[6], 6.67 : Ps_E[7], 10 : Ps_E[9:12], 7.5 : Ps_E[8], 15 : Ps_E[15:18], 13.33 : Ps_E[12:15]}, index=Es)
    df_V = pd.DataFrame(data={1 : V_E[0], 1.33 : V_E[1], 2 : V_E[2],  4 : V_E[5], 2.5 : V_E[3], 3.33 : V_E[4], 5 : V_E[6], 6.67 : V_E[7], 10 : V_E[9:12], 7.5 : V_E[8], 15 : V_E[15:18], 13.33 : V_E[12:15]}, index=Es)
    df_En = pd.DataFrame(data={1 : En_E[0], 1.33 : En_E[1], 2 : En_E[2], 2.5 : En_E[3], 3.33 : En_E[4], 4 : En_E[5], 5 : En_E[6], 6.67 : En_E[7], 7.5 : En_E[8], 10 : En_E[9:12], 13.33 : En_E[12:15], 15 : En_E[15:18]}, index=Es)

    df_Pn = df_Pn.astype(float)
    df_Ps = df_Ps.astype(float)
    df_V = df_V.astype(float)
    df_En = df_En.astype(float)
    
    # Automatically create het maps for Pn, Ps, V and En
    sb.set(font_scale=1.2)
    plt.figure(figsize = (9.5, 4.8))
    sb.heatmap(df_Pn, annot=True, mask=df_Pn.isnull(), cbar_kws={"label": "Pn"})
    plt.xlabel("Csg")
    plt.ylabel("Cng")
    plt.show()

    plt.figure(figsize = (8, 4.8))
    sb.heatmap(df_Ps, annot=True, mask=df_Ps.isnull(), cmap="YlGnBu_r", cbar_kws={"label": "Ps"})
    plt.xlabel("Csg")
    plt.ylabel("Cng")
    plt.show()

    plt.figure(figsize = (10, 4.8))
    sb.heatmap(df_V, annot=True, mask=df_V.isnull(), fmt='.2g', cmap="YlOrBr_r", cbar_kws={"label": "V"})
    plt.xlabel("Csg")
    plt.ylabel("Cng")
    plt.show()
    
    plt.figure(figsize = (8.2, 4.8))
    sb.heatmap(df_En, annot=True, mask=df_En.isnull(), fmt='.0f', cbar_kws={"label": "En"})
    plt.xlabel("Csg")
    plt.ylabel("Cng")
    plt.show()
    
else:
    print("FG has multiple values in its array")

In [ ]:
# 3D plot of ammonia conversion efficicency vs dilution rate vs ratio initial concentration methane:ammonia

# These results are only calculated if one value for FG is given
if len(FG) == 1:

    # Set-up of the plot/figure
    sb.set(font_scale=1.2, style="white")
    fig = plt.figure()
    ax = plt.gca(projection='3d')

    #list of all different graphs (grph turned off by '#' are a double ratio)
    x = Dg[4800:100000]
    y = data_E[1][4800:100000]
    ax.plot(x, y, zs=10, color='g', zdir="y")

    x = Dg[4800:100000]
    y = data_E[3][4800:100000]
    ax.plot(x, y, zs=4, color='k', zdir="y")

    x = Dg[4800:100000]
    y = data_E[5][4800:100000]
    ax.plot(x, y, zs=2, color='r', zdir="y")

    x = Dg[4800:100000]
    y = data_E[7][4800:100000]
    ax.plot(x, y, zs=1.333, color='k', zdir="y")

    x = Dg[4800:100000]
    y = data_E[9][4800:100000]
    ax.plot(x, y, zs=1, color='r', zdir="y")

    x = Dg[4800:100000]
    y = data_E[11][4800:100000]
    ax.plot(x, y, zs=25, color='k', zdir="y")

    #x = Dg[4800:100000]
    #y = data_E[13][4800:100000]
    #ax.plot(x, y, zs=10, color='k', zdir="y")

    x = Dg[4800:1000001]
    y = data_E[15][4800:100000]
    ax.plot(x, y, zs=5, color='r', zdir="y")

    x = Dg[4800:100000]
    y = data_E[17][4800:100000]
    ax.plot(x, y, zs=3.333, color='r', zdir="y")

    x = Dg[4800:100000]
    y = data_E[19][4800:100000]
    ax.plot(x, y, zs=2.5, color='k', zdir="y")

    x = Dg[4800:100000]
    y = data_E[21][4800:100000]
    ax.plot(x, y, zs=50, color='r', zdir="y")

    x = Dg[4800:100000]
    y = data_E[23][4800:100000]
    ax.plot(x, y, zs=20, color='r', zdir="y")

    #x = Dg[4800:100000]
    #y = data_E[25][4800:100000]
    #ax.plot(x, y, zs=10, color='r', zdir="y")

    x = Dg[4800:100000]
    y = data_E[27][4800:100000]
    ax.plot(x, y, zs=6.667, color='k', zdir="y")

    #x = Dg[4800:100000]
    #y = data_E[29][4800:100000]
    #ax.plot(x, y, zs=5, color='r', zdir="y")

    x = Dg[4800:100000]
    y = data_E[31][4800:100000]
    ax.plot(x, y, zs=75, color='k', zdir="y")

    x = Dg[4800:100000]
    y = data_E[33][4800:100000]
    ax.plot(x, y, zs=30, color='r', zdir="y")

    x = Dg[4800:100000]
    y = data_E[35][4800:100000]
    ax.plot(x, y, zs=15, color='k', zdir="y")

    #x = Dg[4800:100000]
    #y = data_E[37][4800:100000]
    #ax.plot(x, y, zs=10, color='g', zdir="y")

    x = Dg[4800:100000]
    y = data_E[39][4800:100000]
    ax.plot(x, y, zs=7.5, color='y', zdir="y")

    x = Dg[4800:100000]
    y = data_E[41][4800:100000]
    ax.plot(x, y, zs=100, color='r', zdir="y")

    x = Dg[4800:100000]
    y = data_E[43][4800:100000]
    ax.plot(x, y, zs=40, color='k', zdir="y")

    #x = Dg[4800:100000]
    #y = data_E[45][4800:100000]
    #ax.plot(x, y, zs=20, color='b', zdir="y")

    x = Dg[4800:100000]
    y = data_E[47][4800:100000]
    ax.plot(x, y, zs=13.333, color='r', zdir="y")

    #x = Dg[4800:100000]
    #y = data_E[49][4800:100000]
    #ax.plot(x, y, zs=10, color='b', zdir="y", )

    # Set axes of the 3D plot
    ax.view_init(25, -55)
    ax.set_xlabel('Dg')
    ax.set_ylabel('En')
    ax.set_zlabel('Ratio')
    # Plot the 3D plot
    plt.show()

else:
    print("FG has multiple values in its array")

In [ ]:
# 3D plot of methane conversion efficicency vs dilution rate vs ratio methane:ammonia

# These results are only calculated if one value for FG is given
if len(FG) == 1:

    # Set-up of the plot/figure
    sb.set(font_scale=1.2, style="white")
    fig = plt.figure()
    ax = plt.gca(projection='3d')

    #list of all different graphs (grph turned off by '#' are a double ratio)
    x = Dg[4800:100000]
    y = data_E[0][4800:100000]
    ax.plot(x, y, zs=10, color='k', zdir="x")

    x = Dg[4800:100000]
    y = data_E[2][4800:100000]
    ax.plot(x, y, zs=4, color='k', zdir="x")

    x = Dg[4800:100000]
    y = data_E[4][4800:100000]
    ax.plot(x, y, zs=2, color='b', zdir="x")

    x = Dg[4800:100000]
    y = data_E[6][4800:100000]
    ax.plot(x, y, zs=1.333, color='k', zdir="x")

    x = Dg[4800:100000]
    y = data_E[8][4800:100000]
    ax.plot(x, y, zs=1, color='b', zdir="x")

    x = Dg[4800:100000]
    y = data_E[10][4800:100000]
    ax.plot(x, y, zs=25, color='k', zdir="x")

    #x = Dg[4800:100000]
    #y = data_E[12][4800:100000]
    #ax.plot(x, y, zs=10, color='k', zdir="y")

    x = Dg[4800:1000001]
    y = data_E[14][4800:100000]
    ax.plot(x, y, zs=5, color='b', zdir="x")

    x = Dg[4800:100000]
    y = data_E[16][4800:100000]
    ax.plot(x, y, zs=3.333, color='b', zdir="x")

    x = Dg[4800:100000]
    y = data_E[18][4800:100000]
    ax.plot(x, y, zs=2.5, color='k', zdir="x")

    x = Dg[4800:100000]
    y = data_E[20][4800:100000]
    ax.plot(x, y, zs=50, color='b', zdir="x")

    x = Dg[4800:100000]
    y = data_E[22][4800:100000]
    ax.plot(x, y, zs=20, color='b', zdir="x")

    #x = Dg[4800:100000]
    #y = data_E[24][4800:100000]
    #ax.plot(x, y, zs=10, color='b', zdir="y")

    x = Dg[4800:100000]
    y = data_E[26][4800:100000]
    ax.plot(x, y, zs=6.667, color='k', zdir="x")

    #x = Dg[4800:100000]
    #y = data_E[28][4800:100000]
    #ax.plot(x, y, zs=5, color='b', zdir="y")

    x = Dg[4800:100000]
    y = data_E[30][4800:100000]
    ax.plot(x, y, zs=75, color='k', zdir="x")

    x = Dg[4800:100000]
    y = data_E[32][4800:100000]
    ax.plot(x, y, zs=30, color='b', zdir="x")

    x = Dg[4800:100000]
    y = data_E[34][4800:100000]
    ax.plot(x, y, zs=15, color='k', zdir="x")

    #x = Dg[4800:100000]
    #y = data_E[36][4800:100000]
    #ax.plot(x, y, zs=10, color='g', zdir="y")

    x = Dg[4800:100000]
    y = data_E[38][4800:100000]
    ax.plot(x, y, zs=7.7, color='b', zdir="x")

    x = Dg[4800:100000]
    y = data_E[40][4800:100000]
    ax.plot(x, y, zs=100, color='b', zdir="x")

    x = Dg[4800:100000]
    y = data_E[42][4800:100000]
    ax.plot(x, y, zs=40, color='k', zdir="x")

    #x = Dg[4800:100000]
    #y = data_E[44][4800:100000]
    #ax.plot(x, y, zs=20, color='b', zdir="y")

    x = Dg[4800:100000]
    y = data_E[46][4800:100000]
    ax.plot(x, y, zs=13.333, color='b', zdir="x")

    #x = Dg[4800:100000]
    #y = data_E[48][4800:100000]
    #ax.plot(x, y, zs=10, color='b', zdir="y", )

    # Set axes of the 3D plot
    ax.view_init(25, 55)
    ax.set_xlabel('Dg')
    ax.set_ylabel('Es')
    ax.set_zlabel('Ratio')

    plt.show()

else:
    print("FG has multiple values in its array")

In [ ]:
# Calculation of the standard deviations and differences between the methane removal efficiency vs dilution rate for initial methane and ammonia concentrations

# These results are only calculated if one value for FG is given
if len(FG) == 1:

    # Creating a dataframe and heatmap
    Es = pd.DataFrame(data={0 : data_E[0], 2 : data_E[2], 4 : data_E[4], 6 : data_E[6], 8 : data_E[8], 10 : data_E[10], 12 : data_E[12], 14 : data_E[14], 16 : data_E[16], 18 : data_E[18], 20 : data_E[20], 22 : data_E[22], 24 : data_E[24], 26 : data_E[26], 28 : data_E[28], 30 : data_E[30], 32 : data_E[32], 34 : data_E[34], 36 : data_E[36], 38 : data_E[38], 40 : data_E[40], 42 : data_E[42], 44 : data_E[44], 46 : data_E[46], 48 : data_E[48]}, index=Dg)
    sb.heatmap(Es)
    EsT = Es.transpose()
    
    # Calculate values
    print("st.dev. max:", np.amax(np.std(EsT)))
    print("st.dev. min:", np.amin(np.std(EsT)))
    print("st.dev. mean:", np.mean(np.std(EsT)))
    print("absolute max difference:", np.amax(np.amax(EsT)-np.amin(EsT)))
    print("absolute mean difference:", np.mean(np.amax(EsT)-np.amin(EsT)))

else:
    print("FG has multiple values in its array")

In [ ]:
# Calculation of the standard deviations and differences between the methane removal efficiency vs dilution rate for ratio initial concentration methane ammonia 10:1

# These results are only calculated if one value for FG is given
if len(FG) == 1:
    
    # Creating a dataframe and heatmap
    En = pd.DataFrame(data={1 : data_E[0], 12 : data_E[12], 24 : data_E[24], 36 : data_E[36], 48 : data_E[48]}, index=Dg)
    sb.heatmap(En)
    EnT = En.transpose()

    # Calculate values
    print("st.dev. max:", np.amax(np.std(EnT)))
    print("st.dev. min:", np.amin(np.std(EnT)))
    print("st.dev. mean:", np.mean(np.std(EnT)))
    print("absolute max difference:", np.amax(np.amax(EnT)-np.amin(EnT)))
    print("absolute mean difference:", np.mean(np.amax(EnT)-np.amin(EnT)))

else:
    print("FG has multiple values in its array")

In [ ]:
# Calculation of the standard deviations and differences between the ammonia removal efficiency vs dilution rate for ratio initial concentration methane ammonia 10:1

# These results are only calculated if one value for FG is given
if len(FG) == 1:
    
    # Creating a dataframe and heatmap
    En = pd.DataFrame(data={1 : data_E[1], 13 : data_E[13], 25 : data_E[25], 37 : data_E[37], 49 : data_E[49]}, index=Dg)
    sb.heatmap(En)
    EnT = En.transpose()

    # Calculate values
    print("st.dev. max:", np.amax(np.std(EnT)))
    print("st.dev. min:", np.amin(np.std(EnT)))
    print("st.dev. mean:", np.mean(np.std(EnT)))
    print("absolute max difference:", np.amax(np.amax(EnT)-np.amin(EnT)))
    print("absolute mean difference:", np.mean(np.amax(EnT)-np.amin(EnT)))

else:
    print("FG has multiple values in its array")

In [ ]:
# Calculation of the standard deviations and differences between the methane removal efficiency vs dilution rate for ratio initial concentration methane ammonia 5:1

# These results are only calculated if one value for FG is given
if len(FG) == 1:

    # Creating a dataframe and heatmap
    En = pd.DataFrame(data={14 : data_E[14], 28 : data_E[28]}, index=Dg)
    sb.heatmap(En)
    EnT = En.transpose()

    # Calculate values
    print("st.dev. max:", np.amax(np.std(EnT)))
    print("st.dev. min:", np.amin(np.std(EnT)))
    print("st.dev. mean:", np.mean(np.std(EnT)))
    print("absolute max difference:", np.amax(np.amax(EnT)-np.amin(EnT)))
    print("absolute mean difference:", np.mean(np.amax(EnT)-np.amin(EnT)))

else:
    print("FG has multiple values in its array")

In [ ]:
# Calculation of the standard deviations and differences between the ammonia removal efficiency vs dilution rate for ratio initial concentration methane ammonia 5:1

# These results are only calculated if one value for FG is given
if len(FG) == 1:

    # Creating a dataframe and heatmap
    En = pd.DataFrame(data={15 : data_E[15], 29 : data_E[29]}, index=Dg)
    sb.heatmap(En)
    EnT = En.transpose()

    # Calculate values
    print("st.dev. max:", np.amax(np.std(EnT)))
    print("st.dev. min:", np.amin(np.std(EnT)))
    print("st.dev. mean:", np.mean(np.std(EnT)))
    print("absolute max difference:", np.amax(np.amax(EnT)-np.amin(EnT)))
    print("absolute mean difference:", np.mean(np.amax(EnT)-np.amin(EnT)))

else:
    print("FG has multiple values in its array")

In [ ]:
# Calculation of the standard deviations and differences between the methane removal efficiency vs dilution rate for ratio initial concentration methane ammonia 20:1

# These results are only calculated if one value for FG is given
if len(FG) == 1:

    # Creating a dataframe and heatmap
    En = pd.DataFrame(data={22 : data_E[22], 44 : data_E[44]}, index=Dg)
    sb.heatmap(En)
    EnT = En.transpose()

    # Calculate values
    print("st.dev. max:", np.amax(np.std(EnT)))
    print("st.dev. min:", np.amin(np.std(EnT)))
    print("st.dev. mean:", np.mean(np.std(EnT)))
    print("absolute max difference:", np.amax(np.amax(EnT)-np.amin(EnT)))
    print("absolute mean difference:", np.mean(np.amax(EnT)-np.amin(EnT)))

else:
    print("FG has multiple values in its array")

In [ ]:
# Calculation of the standard deviations and differences between the ammonia removal efficiency vs dilution rate for ratio initial concentration methane ammonia 20:1

# These results are only calculated if one value for FG is given
if len(FG) == 1:

    # Creating a dataframe and heatmap
    En = pd.DataFrame(data={23 : data_E[23], 45 : data_E[45]}, index=Dg)
    sb.heatmap(En)
    EnT = En.transpose()

    # Calculate values
    print("st.dev. max:", np.amax(np.std(EnT)))
    print("st.dev. min:", np.amin(np.std(EnT)))
    print("st.dev. mean:", np.mean(np.std(EnT)))
    print("absolute max difference:", np.amax(np.amax(EnT)-np.amin(EnT)))
    print("absolute mean difference:", np.mean(np.amax(EnT)-np.amin(EnT)))

else:
    print("FG has multiple values in its array")

# Plotting results: constant initial conditions, ranging gas flow rate

In [ ]:
# Direct results of the model: initial concentrations vs reactor volume

# These results are only calculated if one value for FG is given
if len(CSG0) == 1 and len(CNG0) == 1:
    
    # plot the results: Csg (methane inital concentration)
    plt.plot(V,data_Cg[0],'b-', label='Fg=100')
    plt.plot(V,data_Cg[2],'r-.', label='Fg=250')
    plt.plot(V,data_Cg[4],'g--', label='Fg=500')
    plt.plot(V,data_Cg[6],'k:', label='Fg=750')
    plt.plot(V,data_Cg[8],'y-', label='Fg=1000')
    plt.ylabel('Csg(V)')
    plt.xlabel('V')
    plt.legend()
    plt.show()

    # plot the results: Cng (ammonia initial concentration)
    plt.plot(V,data_Cg[1],'b-', label='Fg=100')
    plt.plot(V,data_Cg[3],'r-.', label='Fg=250')
    plt.plot(V,data_Cg[5],'g--', label='Fg=500')
    plt.plot(V,data_Cg[7],'k:', label='Fg=750')
    plt.plot(V,data_Cg[9],'y-', label='Fg=1000')
    plt.ylabel('Cng(V)')
    plt.xlabel('V')
    plt.legend()
    plt.show()

else:
    print("CSG0 and/or CNG0 have multiple values in their array")

In [ ]:
# Results sensitivity analysis: Methane

# These results are only calculated if one value for FG is given
if len(CSG0) == 1 and len(CNG0) == 1:
    
    # Styling of the graphs
    sb.set(font_scale=1.2, style="whitegrid")

    # plot the results: Ps vs Dg (methane productivity vs dilution rate)
    plt.plot(data_Dg[0],data_P[0],'k-', label='Fg=2400')
    plt.plot(data_Dg[1],data_P[2],'r-.', label='Fg=4800')
    plt.plot(data_Dg[2],data_P[4],'g--', label='Fg=12000')
    plt.plot(data_Dg[3],data_P[6],'k:', label='Fg=30000')
    plt.plot(data_Dg[4],data_P[8],'y-', label='Fg=60000')
    plt.ylabel('Ps')
    plt.xlabel('Dg')
    plt.xlim(0,10)
    plt.legend()
    plt.show()

    # plot the results: Es vs Dg (methane efficiency vs dilution rate)
    plt.plot(data_Dg[0],data_E[0],'b-', label='Fg=2400')
    plt.plot(data_Dg[1],data_E[2],'k-', label='Fg=4800')
    plt.plot(data_Dg[2],data_E[4],'g--', label='Fg=12000')
    plt.plot(data_Dg[3],data_E[6],'k', label='Fg=30000')
    plt.plot(data_Dg[4],data_E[8],'y-', label='Fg=60000')
    plt.ylabel('η')
    plt.xlabel('Dg')
    plt.xlim(0,10)
    plt.legend()
    plt.show()

else:
    print("CSG0 and/or CNG0 have multiple values in their array")

In [ ]:
# Results sensitivity analysis: Ammonia

# These results are only calculated if one value for FG is given
if len(CSG0) == 1 and len(CNG0) == 1:

    # Styling of the graphs
    sb.set(font_scale=1.2, style="whitegrid")

    # plot the results: Pn vs Dg (ammonia productivity vs dilution rate)
    plt.plot(data_Dg[0],data_P[1],'k-', label='Fg=2400')
    plt.plot(data_Dg[1],data_P[3],'r-.', label='Fg=4800')
    plt.plot(data_Dg[2],data_P[5],'g--', label='Fg=12000')
    plt.plot(data_Dg[3],data_P[7],'k:', label='Fg=30000')
    plt.plot(data_Dg[4],data_P[9],'y-', label='Fg=60000')
    plt.ylabel('Pn')
    plt.xlabel('Dg')
    plt.xlim(0,10)
    plt.legend()
    plt.show()

    # plot the results: En vs Dg (ammonia efficiency vs dilution rate)
    plt.plot(data_Dg[0],data_E[1],'b-', label='Fg=2400')
    plt.plot(data_Dg[1],data_E[3],'k-', label='Fg=4800')
    plt.plot(data_Dg[2],data_E[5],'g--', label='Fg=12000')
    plt.plot(data_Dg[3],data_E[7],'k:', label='Fg=30000')
    plt.plot(data_Dg[4],data_E[9],'y-', label='Fg=60000')
    plt.ylabel('η')
    plt.xlabel('Dg')
    plt.xlim(0,10)
    plt.legend()
    plt.show()
    
else:
    print("CSG0 and/or CNG0 have multiple values in their array")

In [ ]:
# Heat maps of Ps (methane productivity), Pn (ammonia productivity), V (reactor volume) and En (ammonia efficiency)
# against Csg (methane) and Cng (methane initial concentration) ratio >> based on different values of Es (methane efficiency)

# These results are only calculated if one value for FG is given
if len(CSG0) == 1 and len(CNG0) == 1:

    Es = [40, 60, 80] # array of baseline Es
    # Setting up empty arrays
    Ps_E = []
    Pn_E = []
    V_E = []
    En_E = []

    for i in range(0, 9, 2):
        # Calculate the V at the specific Es; with this V calculate Ps, Pn and En, for cases all three Es can be reached
        if np.amax(data_E[i]) > 80 :
            for j in Es:
                Es_func = interp1d(data_E[i], V)
                VE = Es_func(j)
                V_E.append(VE/1e4)

                Ps_func = interp1d(V, data_P[i])
                PsE = Ps_func(VE)
                Ps_E.append(PsE)

                Pn_func = interp1d(V, data_P[i+1])
                PnE = Pn_func(VE)
                Pn_E.append(PnE)

                En_func = interp1d(V, data_E[i+1])
                EnE = En_func(VE)
                En_E.append(EnE)
        # Calculate the V at the specific Es; with this V calculate Ps, Pn and En, for cases 40 and 60% Es can be reached
        elif np.amax(data_E[i]) > 60 :
            for j in Es:
                if j == 80:
                    V_E.append(np.nan)
                    Ps_E.append(np.nan)
                    Pn_E.append(np.nan)
                    En_E.append(np.nan)
                else:    
                    Es_func = interp1d(data_E[i], V)
                    VE = Es_func(j)
                    V_E.append(VE/1e4)

                    Ps_func = interp1d(V, data_P[i])
                    PsE = Ps_func(VE)
                    Ps_E.append(PsE)

                    Pn_func = interp1d(V, data_P[i+1])
                    PnE = Pn_func(VE)
                    Pn_E.append(PnE)

                    En_func = interp1d(V, data_E[i+1])
                    EnE = En_func(VE)
                    En_E.append(EnE)
        # Calculate the V at the specific Es; with this V calculate Ps, Pn and En, for cases all three40% Es can be reached
        else:
            for j in Es:
                if j != 40:
                    V_E.append(np.nan)
                    Ps_E.append(np.nan)
                    Pn_E.append(np.nan)
                    En_E.append(np.nan)
                else:    
                    Es_func = interp1d(data_E[i], V)
                    VE = Es_func(j)
                    V_E.append(VE/1e4)

                    Ps_func = interp1d(V, data_P[i])
                    PsE = Ps_func(VE)
                    Ps_E.append(np.around(PsE, 2))

                    Pn_func = interp1d(V, data_P[i+1])
                    PnE = Pn_func(VE)
                    Pn_E.append(np.around(PnE, 2))

                    En_func = interp1d(V, data_E[i+1])
                    EnE = En_func(VE)
                    En_E.append(EnE)            

    # Creating a dataframe out of Pn, Ps, V and En
    df_Pn = pd.DataFrame(data={FG[0] : Pn_E[0:3], FG[1] : Pn_E[3:6], FG[2] : Pn_E[6:9], FG[3] : Pn_E[9:12], FG[4] : Pn_E[12:15] }, index=Es)
    df_Ps = pd.DataFrame(data={FG[0] : Ps_E[0:3], FG[1] : Ps_E[3:6], FG[2] : Ps_E[6:9], FG[3] : Ps_E[9:12], FG[4] : Ps_E[12:15]}, index=Es)
    df_V = pd.DataFrame(data={FG[0] : V_E[0:3], FG[1] : V_E[3:6], FG[2] : V_E[6:9], FG[3] : V_E[9:12], FG[4] : V_E[12:15]}, index=Es)
    df_En = pd.DataFrame(data={FG[0] : En_E[0:3], FG[1] : En_E[3:6], FG[2] : En_E[6:9], FG[3] : En_E[9:12], FG[4] : En_E[12:15]}, index=Es)

    df_Pn = df_Pn.loc[::-1].astype(float)
    df_Ps = df_Ps.loc[::-1].astype(float)
    df_V = df_V.loc[::-1].astype(float)
    df_En = df_En.loc[::-1].astype(float)
    
    # Automatically create het maps for Pn, Ps, V and En
    sb.set(font_scale=1.2)
    sb.heatmap(df_Pn, annot=True, fmt='.2g', cbar_kws={"label": "Pn"})
    plt.xlabel("Fg")
    plt.ylabel("Es")
    plt.show()
    
    sb.set(font_scale=1.2)
    sb.heatmap(df_Ps, annot=True, cmap="YlGnBu_r", cbar_kws={"label": "Ps"})
    plt.xlabel("Fg")
    plt.ylabel("Es")
    plt.show()
    
    sb.set(font_scale=1.2)
    sb.heatmap(df_V, annot=True, fmt='.2g', cmap="YlOrBr_r", cbar_kws={"label": "V"})
    plt.xlabel("Fg")
    plt.ylabel("Es")
    plt.show()
    
    sb.set(font_scale=1.2)
    sb.heatmap(df_En, annot=True, fmt='.0f', cbar_kws={"label": "En"})
    plt.xlabel("Fg")
    plt.ylabel("Es")
    plt.show()
    
    print(df_Ps)
    
else:
    print("FG has multiple values in its array")

In [ ]:
# Calculation of the standard deviations and differences between the methane removal efficiency vs dilution rate for initial methane and ammonia concentrations

# These results are only calculated if one value for FG is given
if len(CSG0) == 1 and len(CNG0) == 1:

    # Creating a dataframe and heatmap
    Es = pd.DataFrame(data={FG[0] : (data_E[0]*data_Dg[0]/data_Dg[0]), FG[1] : (data_E[2]*data_Dg[1]/data_Dg[0]), FG[2] : (data_E[4]*data_Dg[2]/data_Dg[0]), FG[3] : (data_E[6]*data_Dg[3]/data_Dg[0]), FG[4] : (data_E[8]*data_Dg[4]/data_Dg[0])}, index=data_Dg[0])
    sb.heatmap(Es)
    EsT = Es.transpose()
    
    # Calculate values
    print("st.dev. max:", np.amax(np.std(EsT)))
    print("st.dev. min:", np.amin(np.std(EsT)))
    print("st.dev. mean:", np.mean(np.std(EsT)))
    print("absolute max difference:", np.amax(np.amax(EsT)-np.amin(EsT)))
    print("absolute mean difference:", np.mean(np.amax(EsT)-np.amin(EsT)))

else:
    print("FG has multiple values in its array")

In [ ]:
# Calculation of the standard deviations and differences between the ammonia removal efficiency vs dilution rate for initial methane and ammonia concentrations

# These results are only calculated if one value for FG is given
if len(CSG0) == 1 and len(CNG0) == 1:

    # Creating a dataframe and heatmap
    En = pd.DataFrame(data={FG[0] : (data_E[1]*data_Dg[0]/data_Dg[0]), FG[1] : (data_E[3]*data_Dg[1]/data_Dg[0]), FG[2] : (data_E[5]*data_Dg[2]/data_Dg[0]), FG[3] : (data_E[7]*data_Dg[3]/data_Dg[0]), FG[4] : (data_E[9]*data_Dg[4]/data_Dg[0])}, index=data_Dg[0])
    sb.heatmap(En)
    EnT = En.transpose()

    # Calculate values
    print("st.dev. max:", np.amax(np.std(EnT)))
    print("st.dev. min:", np.amin(np.std(EnT)))
    print("st.dev. mean:", np.mean(np.std(EnT)))
    print("absolute max difference:", np.amax(np.amax(EnT)-np.amin(EnT)))
    print("absolute mean difference:", np.mean(np.amax(EnT)-np.amin(EnT)))

else:
    print("FG has multiple values in its array")